In [ ]:
import numpy as np
import scipy
import pandas as pd
import requests
from os import makedirs, path, listdir, remove
from bs4 import BeautifulSoup, SoupStrainer
import zipfile as zpf
from shutil import rmtree
import matplotlib.pyplot as plt
from scipy.spatial.distance import squareform, pdist, cosine
from sklearn.metrics.pairwise import cosine_similarity
from scipy.optimize import minimize
from matplotlib import cm
import re

import httplib2
import geopandas as gpd
from tqdm import tqdm

In [ ]:
# Load map file
folder='tmp'
london_boroughs_gdf = gpd.read_file(path.join(folder, "london_boroughs_coordinates.shp"))
london_gdf = london_boroughs_gdf.dissolve()
print(london_boroughs_gdf.shape)
london_boroughs_gdf.plot()
plt.show()

In [ ]:
# Load LAQN metadata
london_sites_gdf = gpd.read_file(path.join(folder, "LAQN_sites.shp"))
print(london_sites_gdf.shape)

## Load Data

In [ ]:
# Load LAQN data

NO2_raw = pd.read_csv("tmp/LAQN_NO2_1996-01-01_2021-01-01.csv")
CO_raw = pd.read_csv("tmp/LAQN_CO_1996-01-01_2021-01-01.csv")
O3_raw = pd.read_csv("tmp/LAQN_O3_1996-01-01_2021-01-01.csv")
SO2_raw = pd.read_csv("tmp/LAQN_SO2_1996-01-01_2021-01-01.csv")
PM10_raw = pd.read_csv("tmp/LAQN_PM10_1996-01-01_2021-01-01.csv")
PM25_raw = pd.read_csv("tmp/LAQN_PM25_1996-01-01_2021-01-01.csv")

## Analyse Data

In [ ]:
PM10_raw

In [ ]:
PM10_raw.to_csv('incomplete_PM10_hourly.csv')

In [ ]:
print(NO2_raw.shape)
print(CO_raw.shape)

In [ ]:
def format_dataframe(df, species, freq='D'):
    df = df.copy()
    df['date'] = pd.to_datetime(df.date)
    df = df.groupby(pd.Grouper(key="date", freq='D')).mean()
    df = df.reset_index(level=0)
    df['species'] = species
    df = df[ ['date', 'species'] + [ col for col in df.columns if col not in ['date', 'species'] ] ]
#     df = df.groupby(pd.Grouper(key="date", freq='D')).mean()
#     df = df.reset_index(level=0)
    return df

NO2_test = format_dataframe(NO2_raw, 'NO2')
CO_test = format_dataframe(CO_raw, 'CO')
O3_test = format_dataframe(O3_raw, 'O3')
SO2_test = format_dataframe(SO2_raw, 'SO2')
PM10_test = format_dataframe(PM10_raw, 'PM10')
PM25_test = format_dataframe(PM25_raw, 'PM2.5')

In [ ]:
print(NO2_test.shape)
print(CO_test.shape)
print(O3_test.shape)
print(SO2_test.shape)
print(PM10_test.shape)
print(PM25_test.shape)

In [ ]:
plt.figure(figsize=(8, 3))
species = ['NO2', 'CO', 'O3', 'SO2', 'PM2.5', 'PM10']
stations = [203, 45, 58, 55, 46, 176]
plt.bar(species, stations, edgecolor='black')
plt.title('Stations present in LAQN data per species')
plt.xlabel('Species')
plt.ylabel('No. of stations')

In [ ]:
def compare_stations(df1, df2):
    df1_stations = set(df1.columns.values[1:])
    df2_stations = set(df2.columns.values[1:])
    total = df1_stations.union(df2_stations)
    print(f'df1 stations: {len(df1_stations)}')
    print(f'df2 stations: {len(df2_stations)}')
    print(f'total stations: {len(total)}')
    
compare_stations(NO2_raw, PM10_raw)

In [ ]:
full_dataset = NO2_test.merge(CO_test, 'outer')
full_dataset = full_dataset.merge(O3_test, 'outer')
full_dataset = full_dataset.merge(SO2_test, 'outer')
full_dataset = full_dataset.merge(PM10_test, 'outer')
full_dataset = full_dataset.merge(PM25_test, 'outer')
full_dataset

In [ ]:
def group_dataframe(df, freq='M', reset=False):
    grouped_df = df.copy() 
    if reset:
        grouped_df = grouped_df.reset_index(level=0)
    grouped_df['date'] = pd.to_datetime(grouped_df.date)
    grouped_df = grouped_df.groupby(pd.Grouper(key="date", freq=freq)).mean()
    return grouped_df

def plot_station_data(df, station):
    NO2   = group_dataframe(df[df['species'] == 'NO2'])
    CO    = group_dataframe(df[df['species'] == 'CO'])
    O3    = group_dataframe(df[df['species'] == 'O3'])
    SO2   = group_dataframe(df[df['species'] == 'SO2'])
    PM25  = group_dataframe(df[df['species'] == 'PM25'])
    PM10  = group_dataframe(df[df['species'] == 'PM10'])

    fig, axs = plt.subplots(3, 2, figsize=(15, 12))
    fig.suptitle(f'STATION - {station}', fontsize=20)
    axs[0, 0].plot(NO2.index.values, NO2[station].values)
    axs[0, 0].set_title('NO$_{2}$')
    axs[0, 1].plot(CO.index.values, CO[station].values)
    axs[0, 1].set_title('CO')
    axs[1, 0].plot(O3.index.values, O3[station].values)
    axs[1, 0].set_title('O$_{3}$')
    axs[1, 1].plot(SO2.index.values, SO2[station].values)
    axs[1, 1].set_title('SO$d_{2}$')
    axs[2, 0].plot(PM25.index.values, PM25[station].values)
    axs[2, 0].set_title('PM$_{2.5}$')
    axs[2, 1].plot(PM10.index.values, PM10[station].values)
    axs[2, 1].set_title('PM$_{10}$')
#     fig.subplots_adjust(bottom = 0.5)
    fig.tight_layout()
    
    for ax in axs.flat:
        ax.set(xlabel='date', ylabel='Concentration (µg/m$^3$)')

In [ ]:
full_dataset

In [ ]:
station_list = full_dataset.columns[2:].to_numpy()
sampled = np.random.choice(station_list, 10, replace=False)
for station in sampled:
    plot_station_data(full_dataset, station)
    
'''
Stations with lot of species data: BX1
'''

## Control Dataset Access

In [ ]:
def select_species(df, species, freq='D'):
    data = df.copy()
    data = data[data['species'] == species]
    grouped_data = group_dataframe(data, freq)
    return grouped_data

# Test
test_data = select_species(full_dataset, 'NO2')
test_data

## Histogram of missing data

In [ ]:
# Histogram of missing data and stations

def missing_data(df):
    species_list = ['NO2', 'CO', 'O3', 'SO2', 'PM2.5', 'PM10']
    for i, species in enumerate(species_list):
        data = select_species(df, species)
        print(f'Species: {species}')
        print(f'Total: {data.size}')
        print(f'Missing: {data.isna().sum().sum()}')
        print(f'Proportion available: {100 * data.isna().sum().sum()/data.size}')

def plot_missing_data(df):
    species_list = ['NO2', 'CO', 'O3', 'SO2', 'PM2.5', 'PM10']
    species_list2 = ['NO_2', 'CO', 'O_3', 'SO_2', 'PM_{2.5}', 'PM_{10}']
    
    fig, axs = plt.subplots(3, 2, figsize=(20, 15))
    fig.suptitle('Histogram of missing data', fontsize=24)
    for i, species in enumerate(species_list):
        row, col = i // 2, i % 2

        data = select_species(df, species)
        total = len(data)
        num_missing = data.isna().sum()
        
        
        s2 = species_list2[i]
        axs[row, col].hist(num_missing / total, bins=10, edgecolor = "black")
        axs[row, col].set_title(r'$ %s $' % species_list2[i], fontsize=20)
        fig.tight_layout()

        for ax in axs.flat:
#             ax.set(xlabel='Proportion of missing data', ylabel='No. of stations')
            ax.set_xlabel('Proportion of missing data', fontsize=16)
            ax.set_ylabel('No. of stations', fontsize=16)
            ax.set_ylim([0, 200])
            ax.tick_params(axis='both', which='major', labelsize=14)
        
#         plt.hist(num_missing / total, bins=10, edgecolor = "black")
#         plt.title(species)
#         plt.xlabel('Proportion of missing data')
#         plt.ylabel('No. of stations')

def plot_missing_data2(df):
    species_list = ['NO2', 'CO', 'O3', 'SO2', 'PM2.5', 'PM10']
    species_list2 = ['NO_2', 'CO', 'O_3', 'SO_2', 'PM_{2.5}', 'PM_{10}']
    
    fig, axs = plt.subplots(6, figsize=(20, 15))
    fig.suptitle('Histogram of missing data', fontsize=24)
    for i, species in enumerate(species_list):

        data = select_species(df, species)
        total = len(data)
        num_missing = data.isna().sum()
        
        
        s2 = species_list2[i]
        axs[i].hist(num_missing / total, bins=10, edgecolor = "black")
        axs[i].set_title(r'$ %s $' % species_list2[i], fontsize=20)
        fig.tight_layout()

        for ax in axs.flat:
#             ax.set(xlabel='Proportion of missing data', ylabel='No. of stations')
            ax.set_xlabel('Proportion of missing data', fontsize=16)
            ax.set_ylabel('No. of stations', fontsize=16)
            ax.set_ylim([0, 200])
            ax.tick_params(axis='both', which='major', labelsize=14)
        
#         plt.hist(num_missing / total, bins=10, edgecolor = "black")
#         plt.title(species)
#         plt.xlabel('Proportion of missing data')
#         plt.ylabel('No. of stations')

In [ ]:
species_list = ['NO2', 'CO', 'O3', 'SO2', 'PM2.5', 'PM10']
for species in species_list:
    if species != 'CO':
        m = re.search(r"\d", species).start()
    else:
        m = len(species)
    print(species[:m], species[m:])

In [ ]:
missing_data(full_dataset)

In [ ]:
plot_missing_data(full_dataset)

In [ ]:
PM10_test

In [ ]:
plt.subplots(figsize=(10, 5))
data = select_species(full_dataset, 'NO2')
# data = NO2_test
total = len(data)
num_missing = data.isna().sum()
plt.hist((num_missing / total)*100, bins=10, edgecolor = "black")
plt.title(r'$NO_2$', fontsize=20)
plt.xlabel('Proportion of missing data (%)', fontsize=18)
plt.ylabel('No. of stations', fontsize=18)
plt.xticks(fontsize=16)
plt.yticks(fontsize=16)

In [ ]:
plt.subplots(figsize=(10, 5))
data = select_species(full_dataset, 'O3')
# data = O3_test
total = len(data)
num_missing = data.isna().sum()
plt.hist((num_missing / total)*100, bins=10, edgecolor = "black")
plt.title(r'$CO$', fontsize=20)
plt.xlabel('Proportion of missing data (%)', fontsize=18)
plt.ylabel('No. of stations', fontsize=18)
plt.xticks(fontsize=16)
plt.yticks(fontsize=16)

## Pre Graph Propagation Results

In [ ]:
%run functions.ipynb

In [ ]:
class Dataset():
    def __init__(self, csv_file):
        self.df = pd.read_csv(csv_file)
        self.orig = self.df.copy()
        self.df['date'] = pd.to_datetime(self.df.date)
        
    def drop_null(self, nan_percent):
        # drop column if proportion of NaN elements exceed the nan_percent
        min_count = int(((100-nan_percent)/100)*self.df.shape[0] + 1)
        return self.df.dropna(axis=1, thresh=min_count) 
        
    def fill_mean(self):
        return self.df.fillna(self.df.mean())
    
    def group(self, freq):
        # group the data by the specified freq (month/year) and average across this period, then fill NaN values 
        df = self.df.groupby(pd.Grouper(key="date", freq=freq)).mean()
        return df
    
    def group_and_fill(self, freq):
        # group the data by the specified freq (month/year) and average across this period, then fill NaN values 
        df = self.df.groupby(pd.Grouper(key="date", freq=freq)).mean()
        return df.ffill().bfill()
    
    def fill(self):
        df = self.df.copy()
        for col in df.columns.drop('date'):
            df[col] = df[col].fillna(df.groupby([df.date.dt.year, df.date.dt.month])[col].transform('mean'))
        return df.ffill().bfill()

In [ ]:
class ComputeAM():
    def __init__(self, df):
        am_shape = (df.shape[1], df.shape[1])
        self.am = pd.DataFrame(np.zeros(shape=am_shape), columns=df.columns, index=df.columns)
    
    def euclidean_dist(self, df):
        # np.linalg.norm(complete['TD0'].values - complete['BG3'].values) #test euclidean distance between two columns
        dist_arr = squareform(pdist(df.transpose()))
        return pd.DataFrame(dist_arr, columns=df.columns.unique(), index=df.columns.unique())
    
    def cosine_dist(self, df):
        dist_arr = cosine_similarity(df.transpose())
        np.fill_diagonal(dist_arr, 0)
        return pd.DataFrame(dist_arr, columns=df.columns.unique(), index=df.columns.unique())
    
    def threshold_euclidean(self, df, threshold):
        for col in df.columns:
#             df.loc[df[col] > threshold, col] = 0
#             df.loc[df[col] < threshold, col] = 1
            df[col] = np.where(df[col]>=threshold, 1, 0)
        np.fill_diagonal(df.values, 0)
        return df
    
    def diagonal_degree(self, df):
        diag_series = np.diag(df.sum())
        degree_mat = pd.DataFrame(diag_series, columns=df.columns.unique(), index=df.columns.unique())
        return degree_mat

## PM10

In [ ]:
# PM10_test = full_dataset[full_dataset['species'] == 'PM10'].drop(columns=['species']).set_index('date').dropna(axis=1, how='all')
PM10_data = select_species(full_dataset, 'PM10').dropna(axis=1, how='all')
test_set, max_cols = get_test_set(PM10_data, 300)

In [ ]:
PM10_data.mean().mean()

In [ ]:
nan_entries, initial, testing = force_gaps(test_set, proportion=0.25, seed=1)
filled_data, euclidean = fill_and_refactor(testing)

In [ ]:
# Optimise alpha
res_alpha = minimize(compute_error, 0.3, args=(1.0, 2, initial, nan_entries, filled_data, euclidean))
print(res_alpha)

### Error Plots

In [ ]:
alpha_range = np.linspace(0.1, 0.5, 50)
threshold_range = np.linspace(0.5, 2.0, 16)
L_range = np.arange(1, 6)

loss = np.zeros((len(alpha_range), len(threshold_range)))
for i, val1 in enumerate(alpha_range): 
    for j, val2 in enumerate(threshold_range):
        val1 = round(val1, 2)
        val2 = round(val2, 2)
        
        # TEST VALUE
        t_hop = 3
        loss[i][j] = compute_error(val1, val2, t_hop, initial, nan_entries, filled_data, euclidean)
        
X, Y = np.meshgrid(threshold_range, alpha_range)

fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(111, projection='3d')

surf = ax.plot_surface(X[:50], Y[:50], loss[:50], cmap='plasma', linewidth=2)
fig.colorbar(surf, shrink=0.5, aspect=5)
ax.set_title('Error against alpha and threshold')
ax.set_xlabel('threshold')
ax.set_ylabel('alpha')
ax.set_zlabel('error')

shape = np.unravel_index(loss.argmin(), loss.shape)
print(f'Threshold: {X[shape]}')
print(f'Alpha: {Y[shape]}')
print(f'Loss: {loss.min()}')

In [ ]:
len(initial)

In [ ]:
err = compute_error(0.2061, 0.5, 2, initial, nan_entries, filled_data, euclidean)
print(err / 2025**0.5)

In [ ]:
# Plot loss against alpha and L (hops)

alpha_range = np.linspace(0.1, 0.5, 50)
threshold_range = np.linspace(0.5, 2.0, 16)
L_range = np.arange(1, 6)

loss = np.zeros((len(alpha_range), len(L_range)))
for i, val1 in enumerate(alpha_range): 
    for j, val2 in enumerate(L_range):
        val1 = round(val1, 2)
        val2 = round(val2, 2)
        
        t_threshold = 0.3698
        loss[i][j] = compute_error(val1, t_threshold, val2, initial, nan_entries, filled_data, euclidean)
        
X, Y = np.meshgrid(L_range, alpha_range)

fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(111, projection='3d')

lim1 = 50
lim2 = 4
surf = ax.plot_surface(X[:lim1, :lim2], Y[:lim1, :lim2], loss[:lim1, :lim2], cmap='plasma', linewidth=2)
fig.colorbar(surf, shrink=0.5, aspect=5)
ax.set_title('Error against alpha and L (hops)')
ax.set_xlabel('L (hops)')
ax.set_ylabel('alpha')
ax.set_zlabel('error')

shape = np.unravel_index(loss.argmin(), loss.shape)
print(f'Hops: {X[shape]}')
print(f'Alpha: {Y[shape]}')
print(f'Loss: {loss.min()}')

In [ ]:
# Plot loss against threshold and L (hops)

alpha_range = np.linspace(0.1, 0.5, 50)
threshold_range = np.linspace(0.2, 2.0, 160)
L_range = np.arange(1, 6)

loss = np.zeros((len(threshold_range), len(L_range)))
for i, val1 in enumerate(threshold_range): 
    for j, val2 in enumerate(L_range):
        val1 = round(val1, 2)
        val2 = round(val2, 2)
        
        t_alpha = 0.1163
        loss[i][j] = compute_error(t_alpha, val1, val2, initial, nan_entries, filled_data, euclidean)
        
X, Y = np.meshgrid(L_range, threshold_range)

fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(111, projection='3d')

lim1 = len(threshold_range)
lim2 = len(L_range)
surf = ax.plot_surface(X[:lim1, :lim2], Y[:lim1, :lim2], loss[:lim1, :lim2], cmap='plasma', linewidth=2)
fig.colorbar(surf, shrink=0.5, aspect=5)
ax.invert_xaxis()
ax.set_title('Error against threshold and L (hops)')
ax.set_xlabel('L (hops)')
ax.set_ylabel('threshold')
ax.set_zlabel('error')

shape = np.unravel_index(loss.argmin(), loss.shape)
print(f'Hops: {X[shape]}')
print(f'Threshold: {Y[shape]}')
print(f'Loss: {loss.min()}')

In [ ]:
np.mean(initial)

In [ ]:
# params = 0.2061, 0.37, 2
rmse_err = compute_error(0.2061, 0.37, 2, initial, nan_entries, filled_data, euclidean, error_type='rmse')
smape_err = compute_error(0.2061, 0.37, 2, initial, nan_entries, filled_data, euclidean, error_type='smape')
print(f'RMSE Error: {rmse_err}')
print(f'SMAPE Error: {smape_err}')

In [ ]:
# params = 0.116, 0.37, 3
rmse_err = compute_error(0.116, 0.37, 3, initial, nan_entries, filled_data, euclidean, error_type='rmse')
smape_err = compute_error(0.116, 0.37, 3, initial, nan_entries, filled_data, euclidean, error_type='smape')
print(f'RMSE Error: {rmse_err}')
print(f'SMAPE Error: {smape_err}')

In [ ]:
# Optimise alpha

t_alpha = 0.116
t_threshold = 0.37
t_hop = 3

plt.figure(1)
alpha_err = []
alpha_range = np.linspace(0.0, 0.6, 101)
for alpha in alpha_range:
    err = compute_error(alpha, t_threshold, t_hop, initial, nan_entries, filled_data, euclidean, error_type='rmse')
    alpha_err.append(err)
plt.plot(alpha_range, alpha_err)
plt.title('RMSE Error', fontsize=18)
plt.xlabel('Alpha', fontsize=14)
plt.ylabel('Error', fontsize=14)

plt.figure(2)
hop_err = []
hop_range = np.arange(1, 6)
for L in hop_range:
    err = compute_error(t_alpha, t_threshold, L, initial, nan_entries, filled_data, euclidean, error_type='rmse')
    hop_err.append(err)
plt.plot(hop_range, hop_err)
plt.title('RMSE Error', fontsize=18)
plt.xlabel('Hops', fontsize=14)
plt.ylabel('Error', fontsize=14)


plt.figure(3)
threshold_err = []
threshold_range = np.linspace(0.1, 2.0, 101)
for threshold in threshold_range:
    err = compute_error(t_alpha, threshold, t_hop, initial, nan_entries, filled_data, euclidean, error_type='rmse')
    threshold_err.append(err)
plt.plot(threshold_range, threshold_err)
plt.title('RMSE Error', fontsize=18)
plt.xlabel('Threshold', fontsize=14)
plt.ylabel('Error', fontsize=14)

alpha_err = np.nan_to_num(alpha_err, nan=np.inf)
print('Alpha error: ', min(alpha_err), alpha_range[np.argmin(alpha_err)])
print('Hops error: ', min(hop_err), hop_range[np.argmin(hop_err)])
print('Threshold error: ', min(threshold_err), threshold_range[np.argmin(threshold_err)])
print(min(alpha_err))

In [ ]:
# OPTIMISED PARAMETERS
pm10_alpha = 0.116
pm10_threshold = 0.37
pm10_hops = 3
# error -> 9.810

Z, A = compute_progation_matrix(filled_data, euclidean, threshold=pm10_threshold, L=pm10_hops, alpha=pm10_alpha)
final = []
for entry in nan_entries:
    final.append(Z[entry])

x = np.arange(200)
plt.figure(figsize=(5, 5))
plt.scatter(initial, final)
plt.plot(x, x, color='black')
plt.title(r'Algorithm evaluation (RMSE = 9.81)')
plt.xlabel(r'True PM$_{10}$ concentration ($\mu g/mm^3$)')
plt.ylabel(r'Propagated PM$_{10}$ concentration ($\mu g/mm^3$)')

### Scaled to full dataset

In [ ]:
full_data, similarity = fill_and_refactor(PM10_data)

In [ ]:
Z, A = compute_progation_matrix(full_data, similarity, threshold=pm10_threshold, L=pm10_hops, alpha=pm10_alpha)

In [ ]:
corrected = np.copy(Z)

In [ ]:
for (i, column) in enumerate(PM10_data):
    for (j, entry) in enumerate(np.asarray(PM10_data[column])): 
        if not np.isnan(entry):
            corrected[j][i] = entry

In [ ]:
#Get similarity matrix from propagated data

corrected_df = pd.DataFrame(corrected, columns=PM10_data.columns.unique(), index=PM10_data.index.unique())
fd1, similarity = fill_and_refactor(corrected_df)
propagated_df = pd.DataFrame(corrected, columns = PM10_data.columns.unique(), index = PM10_data.index.unique())
propagated_df.to_csv('complete_PM10.csv')

### PM10 Plots

In [ ]:
# Get LAQN site codes
region = 'London'
url_sites = f"http://api.erg.kcl.ac.uk/AirQuality/Information/MonitoringSites/GroupName={region}/Json"
               
london_sites = requests.get(url_sites)
sites_df = pd.DataFrame(london_sites.json()['Sites']['Site'])
site_codes = sites_df["@SiteCode"].tolist()
print(len(site_codes))

In [ ]:
# Get sites for each local authority
site_map = {} # map between local authority codes and list of sites belonging to that local authority
location_map = {} # map between local authority codes and local authority names
# local_codes = set(sites_df['@LocalAuthorityCode'].unique()) # 1 - 33
for i in range(1, 34):
    code = str(i)
    location_map[code] = sites_df[sites_df['@LocalAuthorityCode'] == code]['@LocalAuthorityName'].unique()[0]
    res = sites_df[sites_df['@LocalAuthorityCode'] == code]['@SiteCode']
    site_map[code] = []
    for j, site in res.items():
        site_map[code].append(site)

In [ ]:
grouped = PM10_data

In [ ]:
def group_dataframe2(df, freq='M'):
    grouped_df = df.copy() 
    grouped_df = grouped_df.reset_index(level=0)
    grouped_df['date'] = pd.to_datetime(grouped_df.date)
    grouped_df = grouped_df.groupby(pd.Grouper(key="date", freq=freq)).mean()
    return grouped_df

## Time Series Plots (grouped by day)

In [ ]:
dates = propagated_df.index.values
stations = {'LH0', 'GR5', 'BG2', 'KT4'}
while len(stations) < 10:
    sample = np.random.choice(grouped.columns.values[1:], 1)[0]
    stations.add(sample)
    
for index, station in enumerate(stations):
    plt.figure(index, figsize=(12, 4))
    plt.plot(dates, propagated_df[station].values, color='black', linestyle='dotted')
    plt.plot(dates, grouped[station].values, color='black')
    plt.title(f'Station: {station}')
    plt.xlabel('date', fontsize=10)
    plt.ylabel('PM$_{10}$ Concentrations (µg/m$^3$)', fontsize=10)

## Time Series Plots (grouped by week)

In [ ]:
grouped_W = group_dataframe2(grouped, 'W')
propagated_df_W = group_dataframe2(propagated_df, 'W')

In [ ]:
dates = propagated_df_W.index.values
stations = {'LH0', 'GR5', 'BG2', 'KT4'}
while len(stations) < 10:
    sample = np.random.choice(grouped.columns.values[1:], 1)[0]
    stations.add(sample)
    
for index, station in enumerate(stations):
    plt.figure(index, figsize=(12, 4))
    plt.plot(dates, propagated_df_W[station].values, color='black', linestyle='dotted')
    plt.plot(dates, grouped_W[station].values, color='black')
    plt.title(f'Station: {station}')
    plt.xlabel('date', fontsize=10)
    plt.ylabel('PM$_{10}$ Concentrations (µg/m$^3$)', fontsize=10)

## Time Series Plots (grouped by month)

In [ ]:
grouped_M = group_dataframe2(grouped, 'M')
propagated_df_M = group_dataframe2(propagated_df, 'M')

In [ ]:
dates = propagated_df_M.index.values

In [ ]:
years = np.arange(1996, 2021)
months = np.arange(1, 13)

In [ ]:
miss_count = missing_data_count(grouped)

In [ ]:
dates = propagated_df_M.index.values
stations = {'LH0', 'GR5', 'BG2', 'KT4'}
while len(stations) < 10:
    sample = np.random.choice(grouped.columns.values[1:], 1)[0]
    stations.add(sample)
    
for index, station in enumerate(stations):
    plt.figure(2*index, figsize=(12, 4))
    plt.plot(dates, propagated_df_M[station].values, color='black', linestyle='dotted')
    plt.plot(dates, grouped_M[station].values, color='black')
    
    missing_dates = get_missing_dates(grouped, station)
    plt.scatter(missing_dates, propagated_df_M[station][missing_dates].values, marker='o', color='r', s = 3.0)
    
    plt.title(f'Station: {station}')
    plt.xlabel('date', fontsize=10)
    plt.ylabel('PM$_{10}$ Concentrations (µg/m$^3$)', fontsize=10)
    
    plt.figure(2*index+1, figsize=(12, 4))
    plt.scatter(dates, propagated_df_M[station].values, c=miss_count[station].values, marker='o', s=5.0, cmap='viridis')
    plt.title(f'Station: {station}')
    plt.xlabel('date', fontsize=10)
    plt.ylabel('PM$_{10}$ Concentrations (µg/m$^3$)', fontsize=10)
    plt.colorbar()

### Borough Plots

In [ ]:
prop_cycle = [x['color'] for x in plt.rcParams['axes.prop_cycle']]

In [ ]:
dates = propagated_df_M.index.values
for i in range(1, 34):
    code = str(i)
    cols = [site for site in site_map[code] if site in propagated_df_M.columns]
#     for col in cols
    plt.figure(figsize=(10, 4))
    for j, col in enumerate(cols):
        color = prop_cycle[j % len(prop_cycle)]
        plt.plot(dates, grouped_M[col].values, color=color, label=f'{col}', linewidth=1)
        plt.plot(dates, propagated_df_M[col].values, color=color, linestyle='dashed', linewidth=1)
    plt.title(f'{location_map[code]}', fontsize=13)
    plt.ylabel("PM$_{10}$ Concentrations (µg/m$^3$)", fontsize=11)
    plt.xlabel("Date", fontsize=11)
    plt.legend()

### Similarity Plots

In [ ]:
similar_stations = {}
for station in A.columns:
    similar_stations[station] = similarity[station].sort_values(ascending=False)[:5].index.tolist()

In [ ]:
colour_cycle = prop_cycle = [x['color'] for x in plt.rcParams['axes.prop_cycle']]
plotted_stations = set()

i = 1
for station, similars in similar_stations.items():
    plotted_stations.add(station)
    
    plt.figure(i, figsize=(12, 4))
    plt.plot(dates, grouped_M[station].values, color=colour_cycle[0], label=f'{station}', linewidth=1)
    plt.plot(dates, propagated_df_M[station].values, color=colour_cycle[0], linestyle='dashed', linewidth=1)
    for j, similar in enumerate(similars):
        plt.plot(dates, grouped_M[similar].values, color=colour_cycle[j+1], label=f'{similar}', linewidth=1)
        plt.plot(dates, propagated_df_M[similar].values, color=colour_cycle[j+1], linestyle='dashed', linewidth=1)
#         plt.plot(dates, propagated_df_M[similar].values, label=similar, color=colour_cycle[j+1])
    plt.title(f"Stations similar to: '{station}'")
    plt.xlabel('date', fontsize=10)
    plt.ylabel('PM$_{10}$ Concentrations (µg/m$^3$)', fontsize=10)
    plt.legend()
    
    plt.figure(i+1)
    similarity_list = similarity[station]
    london_sites_gdf_sim = london_sites_gdf.copy()
    london_sites_gdf_sim['Similarity'] = np.nan
    for index, sim_val in similarity_list.items():
        london_sites_gdf_sim.loc[london_sites_gdf_sim['@SiteCode'] == index, 'Similarity'] = sim_val
    london_sites_gdf_sim = london_sites_gdf_sim[~london_sites_gdf_sim['Similarity'].isna()]

    plot_on_map(london_sites_gdf_sim, london_gdf, data_column='Similarity', colorbar=True,
                title=f"Similarity map: '{station}'", 
                data_markersize=5, fontsize=15,
                map_edge_color="gray", figsize=(15,7), axis="on", mark=station)
    
    
    i += 1
    if i == 40:
        break

In [ ]:
# Load LAQN metadata
london_landuse = gpd.read_file(path.join(folder, "gis_osm_landuse_a_free_1.shp"))
print(london_landuse.shape)

In [ ]:
land_palette = {
    'allotments': '#002fff',
    'cemetery': 'gray',
    'commercial': 'orange',
    'farmland': '#002fff',
    'farmyard': '#002fff',
    'forest': 'green',
    'grass': 'green',
    'heath': 'green',
    'industrial': '#4B0092',
    'meadow': 'green',
    'military': '#4B0092',
    'nature_reserve': 'green',
    'orchard': 'pink',
    'park': 'green',
    'quarry': 'gray',
    'recreation_ground': 'green',
    'residential': '#E3E3E3',
    'retail': 'orange',
    'scrub': 'green',
}
cmap = matplotlib.colors.ListedColormap([color for key, color in land_palette.items()])

london_landuse.plot(figsize=(10,10), column='fclass', legend=True, cmap=cmap, legend_kwds={'loc': 'center right', 'bbox_to_anchor':(1.3,0.5)})

In [ ]:
def plot_on_osm_map(data_geodataframe, map_geodataframe, cmap, figsize=(20,10), colorbar=False, data_column='Similarity', title='LAQN Monitoring Station Distribution', mark=None, similars=None):
    
    base = data_geodataframe.plot(ax=map_geodataframe.plot(figsize=figsize, 
                                           column='fclass',
                                           legend=False,
                                           cmap=cmap,
                                           alpha=0.5,
                                           legend_kwds={'loc': 'center right', 'bbox_to_anchor':(1.3,0.5)}),
                    color='black', marker='x', markersize=75, linewidths=3)
    
    if colorbar:
        colorbar_max = data_geodataframe[data_column].max()
        norm = plt.Normalize(data_geodataframe[data_column].min(), colorbar_max)
        plt.colorbar(plt.cm.ScalarMappable(cmap=None, 
        norm=norm)).set_label(data_column)
        
    if mark:
        marked = data_geodataframe[data_geodataframe['@SiteCode'] == mark]
        marked.plot(ax=base, marker='o', color='black', markersize=100);

    if mark and similar:
        title = f'{title}\n Similar stations: {similars}'
    
    plt.suptitle(title, fontsize=20)
    plt.xlabel('Longitude', fontsize=14)
    plt.ylabel('Latitude', fontsize=14)
    plt.xticks(fontsize=12)
    plt.yticks(fontsize=12)
    plt.axis("on")
    plt.savefig(f'images_PM10/{mark}_similarity.png')
    plt.show()

In [ ]:
# # Get similarity maps

# for station, similars in similar_stations.items():
#     similarity_list = similarity[station]
#     london_sites_gdf_sim = london_sites_gdf.copy()
#     london_sites_gdf_sim['Similarity'] = np.nan
#     for index, sim_val in similarity_list.items():  
#         #ensure current station is most similar
#         if index == station:
#             london_sites_gdf_sim.loc[london_sites_gdf_sim['@SiteCode'] == index, 'Similarity'] = 100
#         else:
#             london_sites_gdf_sim.loc[london_sites_gdf_sim['@SiteCode'] == index, 'Similarity'] = sim_val
        
#     london_sites_gdf_sim = london_sites_gdf_sim[~london_sites_gdf_sim['Similarity'].isna()]
    
#     # MAP N MOST SIMILAR STATIONS
# #     data_count=10
# #     london_sites_gdf_sim = london_sites_gdf_sim.sort_values(by='Similarity', ascending=False)[:data_count]
    
#     # ... OR MAP STATIONS > 0.9*MAX
#     max_similarity = london_sites_gdf_sim.sort_values(by='Similarity', ascending=False).iloc[1]['Similarity']
#     london_sites_gdf_sim = london_sites_gdf_sim.loc[(london_sites_gdf_sim['Similarity'] >= 0.9*max_similarity)]
      
#     similars = london_sites_gdf_sim['@SiteCode'].values
#     similars = np.setdiff1d(similars, station)
#     plot_on_osm_map(london_sites_gdf_sim[:11], london_landuse, cmap, mark=station, title=f'LAQN $PM_{{10}}$ Dataset - Station {station}', similars=similars[:10])


* Stations more dispersed compared to NO2 map
* Compute average distance between similar stations to get geographical comparative measure (by borough, etc?)

# -----------------------------------------------------------------------------------------------------------
# Cross - PM10, NO2 (Global Mean)
# -----------------------------------------------------------------------------------------------------------

## Dataset Concatenation

In [ ]:
# NO2 dataset (df1 complete or incomplete)

# complete_NO2 = pd.read_csv('complete_NO2.csv')
# complete_NO2 = complete_NO2.rename(columns={c: c+'_NO2' for c in complete_NO2.columns if c not in ['date']})
# df1 = complete_NO2

NO2_data = select_species(full_dataset, 'NO2').dropna(axis=1, how='all')
df1 = NO2_data.reset_index()
df1 = df1.rename(columns={c: c+'_NO2' for c in df1.columns if c not in ['date']})

In [ ]:
# PM10 dataset (df2 complete or incomplete)

# complete_PM10 = propagated_df.add_suffix('_PM10')
# df2 = complete_PM10.reset_index()
# df2 = df2.drop(['date'], axis=1)

PM10_data = select_species(full_dataset, 'PM10').dropna(axis=1, how='all')
df2 = PM10_data.reset_index()
df2 = df2.drop(['date'], axis=1)
df2 = df2.add_suffix('_PM10')

In [ ]:
cross_data = pd.concat([df1, df2], axis=1)
cross_data = cross_data.set_index('date')

In [ ]:
cross_data

## Controlled Cross Test Set

In [ ]:
test_set, max_cols = get_test_set(cross_data, 500)

In [ ]:
test_set

In [ ]:
test_set.columns

In [ ]:
test_set.columns[28]

In [ ]:
test = np.arange(test_set.size)
test

In [ ]:
test[np.where(test % test_set.shape[1] > 27)]

In [ ]:
def force_gaps(test_set, proportion=0.25, seed=0):
    np.random.seed(seed)
    testing = test_set.copy()
    
    num_gaps = int(proportion * test_set.size)

    # Replace random entries with NaNs
    num_entries = test_set.size
    entry_list = np.arange(num_entries)
    entry_list = entry_list[np.where(entry_list % test_set.shape[1] > 27)] # gaps only in PM10 data (27 boundary)
    nan_indices = np.random.choice(entry_list, num_gaps, replace=False)
    nan_entries = [(num // test_set.shape[1], num % test_set.shape[1]) for num in nan_indices]

    initial = []
    for entry in nan_entries:
        initial.append(testing.iloc[entry])
        testing.iloc[entry] = np.nan
    return nan_entries, initial, testing

In [ ]:
# Global mean normalisation
missing_prop = 0.10 # VARIABLE
nan_entries, initial, testing = force_gaps(test_set, proportion=missing_prop, seed=1)

In [ ]:
filled_data, euclidean = fill_and_refactor(testing)

In [ ]:
filled_data

### Functions

In [ ]:
def basic_graph_propagation(X, A, w, L, a=0.5, b=0.5):
    D_list = np.sum(A, axis=1) # D matrix
    w = np.array(w) 
    prop_matrix = np.diag(D_list**-a).dot(A).dot(np.diag(D_list**-b)) # DAD^(-1)
    prop_matrix = np.nan_to_num(prop_matrix) # convert NaNs to 0s
    
    pi = np.zeros_like(X)
    r = X
    for i in range(L):
        Y_i = w[i:].sum()
        Y_iplus = w[i+1:].sum()
        
        # update pi estimate
        q = (w[i]/Y_i) * r
        pi += q
        
        # update r
        r = (Y_i/Y_iplus) * prop_matrix.dot(r.T).T
        
    q = w[L]/w[L:].sum() * r
    pi += q
    return pi

In [ ]:
def rmse_error(initial, final):
    return np.linalg.norm(np.array(initial) - np.array(final)) / len(initial)**0.5

def smape_error(initial, final):
    initial, final = np.array(initial), np.array(final)
    num = np.absolute(initial - final)
    den = (np.absolute(initial) + np.absolute(final)) / 2
    elems = num/den
    return np.sum(elems) / elems.size

def compute_error(alpha, threshold, L, initial, nan_entries, data, euclideans, error_type='rmse'):
    prop = GraphPropagation()
    A = prop.threshold_am(euclideans, threshold)
    A.iloc[28:, 28:] = 0
    w = [alpha*(1-alpha)**i for i in range(10)]

    # Apply algorithm
    array_data = data.to_numpy()
    Z = basic_graph_propagation(array_data, A, w, L)
    
    final = []
    for entry in nan_entries:
        final.append(Z[entry])
    
    if error_type == 'rmse':
        error = rmse_error(initial, final)
    elif error_type == 'smape':
        error = smape_error(initial, final)
    
    return error

### Error Plots

### Testing

In [ ]:
alpha_range = np.linspace(0.05, 0.3, 50)
threshold_range = np.linspace(0.5, 2.0, 16)
L_range = np.arange(1, 6)

loss = np.zeros((len(alpha_range), len(threshold_range)))
for i, val1 in enumerate(alpha_range): 
    for j, val2 in enumerate(threshold_range):
        val1 = round(val1, 2)
        val2 = round(val2, 2)
        
        # TEST VALUE
        t_hop = 2
        loss[i][j] = compute_error(val1, val2, t_hop, initial, nan_entries, filled_data, euclidean)
        
X, Y = np.meshgrid(threshold_range, alpha_range)

fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(111, projection='3d')

surf = ax.plot_surface(X[:50], Y[:50], loss[:50], cmap='plasma', linewidth=2)
fig.colorbar(surf, shrink=0.5, aspect=5)
ax.set_title('Error against alpha and threshold')
ax.set_xlabel('threshold')
ax.set_ylabel('alpha')
ax.set_zlabel('error')

shape = np.unravel_index(loss.argmin(), loss.shape)
print(f'Threshold: {X[shape]}')
print(f'Alpha: {Y[shape]}')
print(f'Loss: {loss.min()}')

In [ ]:
filled_data

In [ ]:
err = compute_error(0.22244897959183674, 1.3, 2, initial, nan_entries, filled_data, euclidean)
print(err)

In [ ]:
# Plot loss against alpha and L (hops)

alpha_range = np.linspace(0.1, 0.5, 50)
threshold_range = np.linspace(0.5, 2.0, 16)
L_range = np.arange(1, 6)

loss = np.zeros((len(alpha_range), len(L_range)))
for i, val1 in enumerate(alpha_range): 
    for j, val2 in enumerate(L_range):
        val1 = round(val1, 2)
        val2 = round(val2, 2)
        
        t_threshold = 0.709433962264151
        loss[i][j] = compute_error(val1, t_threshold, val2, initial, nan_entries, filled_data, euclidean)
        
X, Y = np.meshgrid(L_range, alpha_range)

fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(111, projection='3d')

lim1 = 50
lim2 = 4
surf = ax.plot_surface(X[:lim1, :lim2], Y[:lim1, :lim2], loss[:lim1, :lim2], cmap='plasma', linewidth=2)
fig.colorbar(surf, shrink=0.5, aspect=5)
ax.set_title('Error against alpha and L (hops)')
ax.set_xlabel('L (hops)')
ax.set_ylabel('alpha')
ax.set_zlabel('error')

shape = np.unravel_index(loss.argmin(), loss.shape)
print(f'Hops: {X[shape]}')
print(f'Alpha: {Y[shape]}')
print(f'Loss: {loss.min()}')

In [ ]:
# Plot loss against threshold and L (hops)

alpha_range = np.linspace(0.1, 0.5, 50)
threshold_range = np.linspace(0.2, 2.0, 160)
L_range = np.arange(1, 6)

loss = np.zeros((len(threshold_range), len(L_range)))
for i, val1 in enumerate(threshold_range): 
    for j, val2 in enumerate(L_range):
        val1 = round(val1, 2)
        val2 = round(val2, 2)
        
        t_alpha = 0.3776
        loss[i][j] = compute_error(t_alpha, val1, val2, initial, nan_entries, filled_data, euclidean)
        
X, Y = np.meshgrid(L_range, threshold_range)

fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(111, projection='3d')

lim1 = len(threshold_range)
lim2 = len(L_range)
surf = ax.plot_surface(X[:lim1, :lim2], Y[:lim1, :lim2], loss[:lim1, :lim2], cmap='plasma', linewidth=2)
fig.colorbar(surf, shrink=0.5, aspect=5)
ax.invert_xaxis()
ax.set_title('Error against threshold and L (hops)')
ax.set_xlabel('L (hops)')
ax.set_ylabel('threshold')
ax.set_zlabel('error')

shape = np.unravel_index(loss.argmin(), loss.shape)
print(f'Hops: {X[shape]}')
print(f'Threshold: {Y[shape]}')
print(f'Loss: {loss.min()}')

In [ ]:
cross_alpha = 0.3776
cross_threshold = 0.7094
cross_hops = 1

rmse_err = compute_error(cross_alpha, cross_threshold, cross_hops, initial, nan_entries, filled_data, euclidean, error_type='rmse')
smape_err = compute_error(cross_alpha, cross_threshold, cross_hops, initial, nan_entries, filled_data, euclidean, error_type='smape')
print(f'RMSE Error: {rmse_err}')
print(f'SMAPE Error: {smape_err}')

In [ ]:
# OPTIMISED PARAMETERS

Z, A = compute_progation_matrix(filled_data, euclidean, threshold=cross_threshold, L=cross_hops, alpha=cross_alpha)
final = []
for entry in nan_entries:
    final.append(Z[entry])

x = np.arange(100)
plt.figure(figsize=(5, 5))
plt.scatter(initial, final)
plt.plot(x, x, color='black')
plt.title('Algorithm evaluation (RMSE = 9.002)')
plt.xlabel('True pollutant concentration')
plt.ylabel('Propagated pollutant concentration')

### Scaled to full dataset

In [ ]:
full_data, similarity_cross = fill_and_refactor(cross_data)

In [ ]:
similarity_cross

In [ ]:
def get_L(matrix):
    total = np.zeros_like(matrix)
    
    i = 0
    while np.count_nonzero(total) != matrix.size:
        i += 1
        total += np.linalg.matrix_power(matrix, i)
        if i == 10:
            break
    return i

def compute_progation_matrix(data, euclideans, threshold, L=None, alpha=None, w=np.array([1, 0, 0, 0])):
    prop = GraphPropagation()
    A = prop.threshold_am(euclideans, threshold)
    A.iloc[201:, 201:] = 0

    if alpha:
        w = [alpha*(1-alpha)**i for i in range(10)]
    if not L:
        L = get_L(A)

    # Apply algorithm
    array_data = data.to_numpy()
    Z = basic_graph_propagation(array_data, A, w, L)
    return Z, A

In [ ]:
Z, A = compute_progation_matrix(full_data, similarity_cross, threshold=cross_threshold, L=cross_hops, alpha=cross_alpha)

In [ ]:
A

In [ ]:
corrected = np.copy(Z)

In [ ]:
x = y = 0
for (i, column) in enumerate(cross_data):
    for (j, entry) in enumerate(np.asarray(cross_data[column])):
        x += 1
        if not np.isnan(entry):
            y += 1
            corrected[j][i] = entry

In [ ]:
#Get similarity matrix from propagated data

corrected_df = pd.DataFrame(corrected, columns=cross_data.columns.unique(), index=cross_data.index.unique())
fd1, similarity_cross = fill_and_refactor(corrected_df)
cross_propagated_df = pd.DataFrame(corrected, columns = cross_data.columns.unique(), index = cross_data.index.unique())

In [ ]:
cross_propagated_df.to_csv('PM10_from_NO2.csv')

In [ ]:
# Find cross stations with the greatest similarity
top_similarities = (-similarity_cross.stack()).argsort()[:500].values
for ind, i in enumerate(top_similarities):
    idx = similarity_cross.stack().index[i]
    station1 = idx[0]
    station2 = idx[1]
    if (station1.endswith("NO2") and station2.endswith("PM10")):
        print(f'{station1}, {station2}: {similarity_cross[station1][station2]}')
#     print(f'{station1}, {station2}: {similarity_cross[station1][station2]}')

In [ ]:
similarity_cross.mean().mean()

In [ ]:
plt.figure(figsize=(12, 12))

a = np.random.random((16, 16))
# diag_similarity_cross = np.fill_diagonal(A.values, 3)
plt.imshow(A, cmap='magma', interpolation='nearest', vmin=0, vmax=1)
plt.xlabel(r'$NO_2$ and $PM_{10}$ stations')
plt.ylabel(r'$NO_2$ and $PM_{10}$ stations')
plt.colorbar()
plt.show()

In [ ]:
PM10_data.mean().mean()

# Smaller mean value means that station vectors are generally closer to each other (hence bright patches for PM10)

In [ ]:
NO2_data.mean().mean()

In [ ]:
similarity_cross.mean().sort_values()

In [ ]:
grouped = cross_data

In [ ]:
def group_dataframe2(df, freq='M'):
    grouped_df = df.copy() 
    grouped_df = grouped_df.reset_index(level=0)
    grouped_df['date'] = pd.to_datetime(grouped_df.date)
    grouped_df = grouped_df.groupby(pd.Grouper(key="date", freq=freq)).mean()
    return grouped_df

## Time Series Plots (grouped by month)

In [ ]:
propagated_df = cross_propagated_df

In [ ]:
grouped_M = group_dataframe2(grouped, 'M')
propagated_df_M = group_dataframe2(propagated_df, 'M')

In [ ]:
dates = propagated_df_M.index.values

In [ ]:
years = np.arange(1996, 2021)
months = np.arange(1, 13)

In [ ]:
miss_count = missing_data_count(grouped)

In [ ]:
dates = propagated_df_M.index.values
stations = {'ST3_NO2', 'HI1_NO2', 'SK1_PM10', 'ST3_NO2'}
while len(stations) < 10:
    sample = np.random.choice(grouped.columns.values[1:], 1)[0]
    stations.add(sample)
    
for index, station in enumerate(stations):
    plt.figure(2*index, figsize=(12, 4))
    plt.plot(dates, propagated_df_M[station].values, color='black', linestyle='dotted')
    plt.plot(dates, grouped_M[station].values, color='black')
    
    missing_dates = get_missing_dates(grouped, station)
    plt.scatter(missing_dates, propagated_df_M[station][missing_dates].values, marker='o', color='r', s = 3.0)
    
    plt.title(f'Station: {station}')
    plt.xlabel('date', fontsize=10)
    plt.ylabel('PM$_{10}$ Concentrations (µg/m$^3$)', fontsize=10)
    
    plt.figure(2*index+1, figsize=(12, 4))
    plt.scatter(dates, propagated_df_M[station].values, c=miss_count[station].values, marker='o', s=5.0, cmap='viridis')
    plt.title(f'Station: {station}')
    plt.xlabel('date', fontsize=10)
    plt.ylabel('PM$_{10}$ Concentrations (µg/m$^3$)', fontsize=10)
    plt.colorbar()

### Similarity Plots

In [ ]:
similarity = similarity_cross

In [ ]:
similar_stations = {}
for station in A.columns:
    similar_stations[station] = similarity[station].sort_values(ascending=False)[:5].index.tolist()

In [ ]:
colour_cycle = prop_cycle = [x['color'] for x in plt.rcParams['axes.prop_cycle']]
plotted_stations = set()

i = 0
for station, similars in similar_stations.items():
    plotted_stations.add(station)
    
    plt.figure(i, figsize=(12, 4))
    plt.plot(dates, grouped_M[station].values, color=colour_cycle[0], label=f'{station}', linewidth=1)
    plt.plot(dates, propagated_df_M[station].values, color=colour_cycle[0], linestyle='dashed', linewidth=1)
    for j, similar in enumerate(similars):
        plt.plot(dates, grouped_M[similar].values, color=colour_cycle[j+1], label=f'{similar}', linewidth=1)
        plt.plot(dates, propagated_df_M[similar].values, color=colour_cycle[j+1], linestyle='dashed', linewidth=1)
#         plt.plot(dates, propagated_df_M[similar].values, label=similar, color=colour_cycle[j+1])
    plt.title(f"Stations similar to: '{station}'")
    plt.xlabel('date', fontsize=10)
    plt.ylabel('Pollutant Concentrations (µg/m$^3$)', fontsize=10)
    plt.legend()
    
#     plt.figure(i+1)
#     similarity_list = similarity[station]
#     london_sites_gdf_sim = london_sites_gdf.copy()
#     london_sites_gdf_sim['Similarity'] = np.nan
#     for index, sim_val in similarity_list.items():
#         london_sites_gdf_sim.loc[london_sites_gdf_sim['@SiteCode'] == index, 'Similarity'] = sim_val
#     london_sites_gdf_sim = london_sites_gdf_sim[~london_sites_gdf_sim['Similarity'].isna()]

#     plot_on_map(london_sites_gdf_sim, london_gdf, data_column='Similarity', colorbar=True,
#                 title=f"Similarity map: '{station}'", 
#                 data_markersize=5, fontsize=15,
#                 map_edge_color="gray", figsize=(15,7), axis="on", mark=station)
    
    i += 2
    if i == 20:
        break

In [ ]:
# Load LAQN metadata
london_landuse = gpd.read_file(path.join(folder, "gis_osm_landuse_a_free_1.shp"))
print(london_landuse.shape)

### Cross Similarity Plot 

In [ ]:
# Find cross stations with the greatest similarity
top_similarities = (-similarity_cross.stack()).argsort()[:500].values
cross_stations = []
for ind, i in enumerate(top_similarities):
    idx = similarity_cross.stack().index[i]
    station1 = idx[0]
    station2 = idx[1]
    if (station1.endswith("NO2") and station2.endswith("PM10")):
        cross_stations.append(idx)
        print(f'{station1}, {station2}: {similarity_cross[station1][station2]}')
#     print(f'{station1}, {station2}: {similarity_cross[station1][station2]}')

In [ ]:
for i, pair in enumerate(cross_stations):
    plt.figure(i, figsize=(12, 4))
    station1 = pair[0]
    station2 = pair[1]
                    
    plt.plot(dates, propagated_df_M[station1].values, color=colour_cycle[0], label=f'{station1}', linewidth=1)
    plt.plot(dates, propagated_df_M[station2].values, color=colour_cycle[1], label=f'{station2}', linewidth=1)
    
    plt.xlabel('date', fontsize=10)
    plt.ylabel('Pollutant concentrations (µg/m$^3$)', fontsize=10)
    plt.legend()
    
#     plt.plot(dates, propagated_df_M[station1].values, color=colour_cycle[0], linestyle='dashed', linewidth=1)

In [ ]:
print(grouped_M.mean()['CE2_NO2'])
print(grouped_M.mean()['VS1_PM10'])

In [ ]:
grouped_M[station1]

# -----------------------------------------------------------------------------------------------------------
# Cross - PM10, NO2 (Grouped Mean)
# -----------------------------------------------------------------------------------------------------------

In [ ]:
NO2_data = select_species(full_dataset, 'NO2').dropna(axis=1, how='all')
# NO2_norm = NO2_data.divide(NO2_data.mean().mean())
df1_t = NO2_norm.reset_index()
df1_t = df1_t.rename(columns={c: c+'_NO2' for c in df1_t.columns if c not in ['date']})

PM10_data = select_species(full_dataset, 'PM10').dropna(axis=1, how='all')
# PM10_norm = PM10_data.divide(PM10_data.mean().mean())
df2_t = PM10_norm.reset_index()
df2_t = df2_t.drop(['date'], axis=1)
df2_t = df2_t.add_suffix('_PM10')

cross_data2 = pd.concat([df1_t, df2_t], axis=1)
cross_data2 = cross_data2.set_index('date')

In [ ]:
def fill_and_refactor(gap_data):
    filled_data = gap_data.ffill().bfill()
    am = ComputeAM(filled_data)
    euclidean_am = am.euclidean_dist(filled_data) # initially, the larger the value, the more distant and the less similar

    mean = euclidean_am.mean().mean() 
    refactored = (mean / euclidean_am)  # Larger values represent more similar stations
    np.fill_diagonal(refactored.values, 0)
    return filled_data, refactored

In [ ]:
test_set, max_cols = get_test_set(cross_data2, 500)
nan_entries, initial, testing = force_gaps(test_set, proportion=0.25, seed=5)
filled_data, euclidean = fill_and_refactor(testing)

In [ ]:
testing

In [ ]:
# Optimise alpha
res_alpha = minimize(compute_error, 0.3, args=(1.0, 2, initial, nan_entries, filled_data, euclidean))
print(res_alpha)

### Error Plots

In [ ]:
alpha_range = np.linspace(0.1, 0.5, 50)
threshold_range = np.linspace(0.5, 2.0, 16)
L_range = np.arange(1, 6)

loss = np.zeros((len(alpha_range), len(threshold_range)))
for i, val1 in enumerate(alpha_range): 
    for j, val2 in enumerate(threshold_range):
        val1 = round(val1, 2)
        val2 = round(val2, 2)
        
        # TEST VALUE
        t_hop = 2
        loss[i][j] = compute_error(val1, val2, t_hop, initial, nan_entries, filled_data, euclidean)
        
X, Y = np.meshgrid(threshold_range, alpha_range)

fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(111, projection='3d')

surf = ax.plot_surface(X[:50], Y[:50], loss[:50], cmap='plasma', linewidth=2)
fig.colorbar(surf, shrink=0.5, aspect=5)
ax.set_title('Error against alpha and threshold')
ax.set_xlabel('threshold')
ax.set_ylabel('alpha')
ax.set_zlabel('error')

shape = np.unravel_index(loss.argmin(), loss.shape)
print(f'Threshold: {X[shape]}')
print(f'Alpha: {Y[shape]}')
print(f'Loss: {loss.min()}')

In [ ]:
err = compute_error(0.22244897959183674, 1.3, 2, initial, nan_entries, filled_data, euclidean)
print(err)

In [ ]:
# Plot loss against alpha and L (hops)

alpha_range = np.linspace(0.1, 0.5, 50)
threshold_range = np.linspace(0.5, 2.0, 16)
L_range = np.arange(1, 6)

loss = np.zeros((len(alpha_range), len(L_range)))
for i, val1 in enumerate(alpha_range): 
    for j, val2 in enumerate(L_range):
        val1 = round(val1, 2)
        val2 = round(val2, 2)
        
        t_threshold = 1.366
        loss[i][j] = compute_error(val1, t_threshold, val2, initial, nan_entries, filled_data, euclidean)
        
X, Y = np.meshgrid(L_range, alpha_range)

fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(111, projection='3d')

lim1 = 50
lim2 = 4
surf = ax.plot_surface(X[:lim1, :lim2], Y[:lim1, :lim2], loss[:lim1, :lim2], cmap='plasma', linewidth=2)
fig.colorbar(surf, shrink=0.5, aspect=5)
ax.set_title('Error against alpha and L (hops)')
ax.set_xlabel('L (hops)')
ax.set_ylabel('alpha')
ax.set_zlabel('error')

shape = np.unravel_index(loss.argmin(), loss.shape)
print(f'Hops: {X[shape]}')
print(f'Alpha: {Y[shape]}')
print(f'Loss: {loss.min()}')

In [ ]:
# Plot loss against threshold and L (hops)

alpha_range = np.linspace(0.1, 0.5, 50)
threshold_range = np.linspace(0.2, 2.0, 160)
L_range = np.arange(1, 6)

loss = np.zeros((len(threshold_range), len(L_range)))
for i, val1 in enumerate(threshold_range): 
    for j, val2 in enumerate(L_range):
        val1 = round(val1, 2)
        val2 = round(val2, 2)
        
        t_alpha = 0.2224
        loss[i][j] = compute_error(t_alpha, val1, val2, initial, nan_entries, filled_data, euclidean)
        
X, Y = np.meshgrid(L_range, threshold_range)

fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(111, projection='3d')

lim1 = len(threshold_range)
lim2 = len(L_range)
surf = ax.plot_surface(X[:lim1, :lim2], Y[:lim1, :lim2], loss[:lim1, :lim2], cmap='plasma', linewidth=2)
fig.colorbar(surf, shrink=0.5, aspect=5)
ax.invert_xaxis()
ax.set_title('Error against threshold and L (hops)')
ax.set_xlabel('L (hops)')
ax.set_ylabel('threshold')
ax.set_zlabel('error')

shape = np.unravel_index(loss.argmin(), loss.shape)
print(f'Hops: {X[shape]}')
print(f'Threshold: {Y[shape]}')
print(f'Loss: {loss.min()}')

In [ ]:
cross_alpha = 0.22245
cross_threshold = 1.274
cross_hops = 2

rmse_err = compute_error(cross_alpha, cross_threshold, cross_hops, initial, nan_entries, filled_data, euclidean, error_type='rmse')
smape_err = compute_error(cross_alpha, cross_threshold, cross_hops, initial, nan_entries, filled_data, euclidean, error_type='smape')
print(f'RMSE Error: {rmse_err}')
print(f'SMAPE Error: {smape_err}')

# Cross Test Set v1 (old)

In [ ]:
result = pd.concat([df1, df2], axis=1)
result = result.set_index('date')

In [ ]:
#Get dataset

species_comp = ['NO2', 'PM10']
common_cols = np.intersect1d(NO2_raw.columns, PM10_raw.columns).tolist()[:-1] # 147 common stations between NO2 and PM10 datasets

# Select columns of data common to both datasets
PM10_data = select_species(full_dataset, 'PM10')[common_cols]
NO2_data = select_species(full_dataset, 'NO2')[common_cols]

In [ ]:
def get_test_cross_set(df, df2, num_valid_values=500):
    max_size = 0
    max_index = 0

    for i in range(0, df.shape[0], 5):
        test = df.iloc[i:].isnull()
        test.reset_index(drop=True, inplace=True)
        res = test.eq(True).idxmax() # count of consecutive readings per station
        size = res[res > num_valid_values].size # number of stations with over specified number of of readings
        cols = res[res > num_valid_values].keys()

        test = df2[cols].iloc[i:].isnull()
        test.reset_index(drop=True, inplace=True)
        res = test.eq(True).idxmax() # count of consecutive readings per station
        size = res[res > num_valid_values].size # number of stations with over specified number of of readings

        if size > max_size:
            max_size = size
            max_index = i
        
    test = df.iloc[max_index:].isnull()
    test.reset_index(drop=True, inplace=True)
    res = test.eq(True).idxmax()
    max_cols = res[res > num_valid_values].keys()
    test_set = df[max_cols].iloc[max_index:max_index+num_valid_values]

    test = df2[max_cols].iloc[max_index:].isnull()
    test.reset_index(drop=True, inplace=True)
    res = test.eq(True).idxmax()
    max_cols = res[res > num_valid_values].keys()
    test_set2 = df2[max_cols].iloc[max_index:max_index+num_valid_values]
    
    test_set = test_set[max_cols]
    
    return test_set, test_set2, max_cols

In [ ]:
NO2_control, PM10_control, max_cols = get_test_cross_set(NO2_data, PM10_data, num_valid_values=100)

## Cross Correlate

* Normalise dataset by subtracting mean and dividing by standard deviation 

In [ ]:
NO2_control.mean()

In [ ]:
NO2_normalised = NO2_control.sub(NO2_control.mean(), axis='columns').div(NO2_control.std(), axis='columns')
NO2_normalised

In [ ]:
PM10_normalised = PM10_control.sub(PM10_control.mean(), axis='columns').div(PM10_control.std(), axis='columns')
PM10_normalised

In [ ]:
def compute_cross_AM(df1, df2):
    size = df1.columns.size
    column_list = df1.columns.unique()
    cross_AM = pd.DataFrame(np.zeros((size,size)), columns=column_list, index=column_list)
    for column in column_list: # fills rows for single column
        base = df1[column]
        for column2 in column_list: # fills all columns
            compare = df2[column2]
            dist = np.linalg.norm(base - compare)
            cross_AM[column][column2] = dist # cross_AM[select column][select row]
    
    return cross_AM

compute_cross_AM(NO2_normalised, PM10_normalised)